# Homework 3

Домашние задания являются полностью опциональными, оценки никуда не идут.

### Дедлайн: 21.06.20 (вс) 21:00 мск

### Формат сдачи

__Вариант 1__: этот ipynb файл с кодом

дополнительно добавьте:
* "checkpoint" файл из `torch.save(model.state_dict(), ...)` который содержит веса модели для задачи 1, 3 и поможет удостовериться в точности на валидации
* имеется возможность перезапустить весь ipynb файл, чтобы убедиться в финальном результате, например в 1й и 3й задаче (можете сделать конструкцию if else, н: если файл с весами присутствует, тогда показываем точность, иначе запускаем training рутину и выводим в конце точность)

__Вариант 2__:
сдаёте в виде набора .py скриптов (н. train.py, test.py, ...) на каждое задание с инструкцией как запустить и убедиться в полученных вами результатах

# Задача 1 - Train your own model (6 баллов)

В этом задании вы построете сверточную нейросеть (CNN) для решения Tiny ImageNet классификации. Постарайтесь добиться максимальной точности на валидации.

### Оценивание

* За преодоление каждого порога даётся 1 балл
  * 25.0%
  * 30.0%
  * 32.5%
  * 35.0%
  * 37.5%
  * 40.0%
    
### Ограничения

* __Нельзя использовать предобученные нейросети.__ Сами архитектуры использовать можно.

### Советы

* Одно изменение в один момент времени
* Используйте GPU, постарайтесь написать device-agnostic код, чтобы нигде не присутствовало `.cuda()`, а было `.to(device)`
* Логируйте промежуточный результаты, например в TensorBoard
* С чем можно поиграться: optimizer, lr scheduler, архитектура, инициализации, loss, регуляризации (dropout, cutmix, weight decay, аугментации и тд), и кучу всего разного
* Про "фишечки", которые можно внедрить для улучшения качества, можно посмотреть здесь https://arxiv.org/abs/1812.01187
* Постарайтесь по максимуму переиспользовать написанный код, то есть реализовать в виде набора функций например тренировочную рутину и ее использовать в задаче 1, 2, 4
* Как обычно: читабельный, понятный код приветствуется

### Какую архитектуру выбрать для обучения?
* пишем сами
* берём готовую архитектуру (например отсюда https://github.com/rwightman/pytorch-image-models (очень классный ресурс с модными архитектурами) или отсюда https://pytorch.org/docs/stable/torchvision/models.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tiny_imagenet
tiny_imagenet.download(".")

Тренировочные и валидационные данные лежат в `tiny-imagenet-200/train` и `tiny-imagenet-200/val`.

In [ ]:
# put your code here

Когда закончите обучение, сделайте отдельно инференс на валидации и выведите ниже результат.

In [ ]:
val_accuracy = comput_val_acc(...)
print("Validation accuracy: %.2f%%" % (val_accuracy * 100))

### Отчет

Кратако укажите, что вы попробывали, что получилось и дало прирост, а что нет

Сколько параметров у модели?

Как быстро она предсказывает класс для 1й картинки?

Oпцианально: выведите top-1, top-5 точность вашей модели

Опцианально: выведите картинки, на которых ваша модель сильнее всего ошибается (вероятность класса, соответствующая таргету, минимальная)

# Задача 2 - Transfer Learning (1 балл)

Выберите любую предобученную модель. Сделайте transfer learning для датасета из задачи 1. Постарайтесь по-максимуму воспользоваться кодом из задачи 1, чтобы не дублировать его. Выведите точность на валидации.

# Задача 3 - Grouped Convolutions (2 балла)

Задача на понимание групповых сверток.

Стандартный вариант указать параметр __groups__  у `nn.Conv2d` и вызвать стандартный forward.

Попробуйте написать `forward` сами, воспользовавшись `torch.nn.functional.conv2d()` с одним ограничением, что параметр __groups__ в `torch.nn.functional.conv2d()` передавать нельзя.

Для решения советую циклом пройтись по `range(out_channels)` возможно с ненулевым шагом.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def custom_forward(conv, x):
    # put your code here
    return result

conv = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, groups=2, padding=1)
x = torch.rand(1, 8, 16, 16)
assert torch.allclose(conv(x), custom_forward(conv, x))

# Задача 4 - Self-supervised learning (много баллов)

Если вы дошли до задачи 4 и у вас еще много сил, энергии и времени, то вот вам настоящий challenge:

Прочитайте статью про FixMatch https://arxiv.org/abs/2001.07685

Вам предлагается в каком-либо виде воспользоваться предлагаемым в статье новым loss-ом применимо к задаче 1. А именно: в статье предлагается вариант того, как эффективно воспользоваться неразмеченными данными для улучшения качества модели.
Если у вас написан хороший код для задачи 1, то не должно вызвать больших трудностей добавить упомянутый подход.
Смотреть / использовать готовые какие-то реализации разрешается, главное вы должны понимать код, которым воспользовались.

Пара советов:
* скорей всего в одной из лекций мы покроем как писать свой вариант кастомный класс для загрузки данных, но вы можете сейчас попробовать погуглить (pytorch custom dataset) - он вам скорей всего понадобится, чтобы грузить одновременно размеченные и неразмеченные данные в 1 батче
* неразмеченные данные можно взять из любого другого датасета, например cifar-100 либо же для начала можно просто имеющиеся train данные поделить на две части и забыть про существование таргетов на одной из частей
* воспользуйтесь той же самой моделью и теми же самыми параметрами оптимизатора и т.д какие вы выбрали в 1й задаче
* сильные и слабые аугментации (см. статью) можете сделать внутри вашего кастомного класса с датасетом (т.е. агументировать данные до отправки на gpu) либо делать их уже после загрузки батча из dataloader (желательно на гпу). Во втором варианте советую воспользоваться https://kornia.readthedocs.io/en/latest/augmentation.html
* аугментации можете для дз сделать любые свои в качестве слабых и сильных, но на будущее старайтесь реимплемнтировать разные походы из статей как можно более похожим образом для начала, если еще нет интуиции как поменять что-то под вашу задачу, чтобы в случае если вы встраивате метод в какой-то пайплайн и у вас в итоге плохое качество, знать что в этом месте например проблемы нет, значит надо искать в другом месте